In [1]:
# Get timings for each cell
%load_ext autotime

In [2]:
import os
import numpy as np
import pandas as pd

from astropy import units as u
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.stats import sigma_clip, sigma_clipped_stats
from ccdproc import ImageFileCollection

from tqdm import tqdm  # Progress bars

from piaa.utils import helpers
from pong.utils import storage
from piaa.utils.plot import pixel_hist

from matplotlib import pyplot as plt

time: 2 s


### Get the calibration images

Download the files from google storage buckets. 

In [3]:
# Get our calibration images for the unit
calibration_images = storage.get_observation_blobs('PAN001/calibration/')

# Flats are included above but a more recent set was also taken
flat_images = storage.get_observation_blobs('PAN001/flats/')

time: 471 ms


In [4]:
# Save FITS files to this location
local_save_dir = '/var/panoptes/images/flats'
os.makedirs(local_save_dir, exist_ok=True)

time: 1.37 ms


In [5]:
# !!!WARNING - Download from the google storage buckets - lots of data - WARNING!!!
file_list = list()
for fn in tqdm(flat_images + calibration_images):
    local_fn = storage.download_fits_file(fn, save_dir=local_save_dir, unpack=False)
    file_list.append(local_fn)

100%|██████████| 316/316 [00:00<00:00, 30712.55it/s]

time: 14.6 ms


In [8]:
# Get the local path names
bias_list = [fn for fn in file_list if 'bias' in fn]
dark_list = [fn for fn in file_list if 'dark' in fn]
flat_list = [fn for fn in file_list if 'PAN001_flats_14d3bd_20180725T052258' in fn]

time: 1.7 ms


In [82]:
f0 = helpers.get_rgb_data(fits.getdata(flat_list[-2]).astype(np.float) - 2047)
f1 = helpers.get_rgb_data(fits.getdata(flat_list[-1]).astype(np.float) - 2047)

time: 2.14 s


In [83]:
for i, c in zip(range(len(f0)), 'rgb'):
    c0 = f0[i].compressed()
    c1 = f1[i].compressed()
    mean = np.mean(2 * c0)
    var = np.var(c0 - c1)
    print("{} {:9.2f} {:7.2f} = {:.2f}".format(c, mean, var, mean/var))

r   3496.47 2046.32 = 1.71
g  13897.69 8075.13 = 1.72
b  17733.18 9834.57 = 1.80
time: 805 ms
